<a href="https://colab.research.google.com/github/branjbar/nlp-bert-family/blob/master/pytorch/pythorch_roberta_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# CHANGES:
## lr increase
## TRAIN_BATCH_SIZE
## VALID_BATCH_SIZE

In [0]:
# To avoid notebook get disconnected
"""
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button").click() 
}setInterval(ClickConnect,60000)
"""

"""
caffeinate
"""

'\ncaffeinate\n'

In [0]:
# TODOS:
## add tic and toc to calculate the total time!
## check the pretrained model
## remove neutral from the filtering

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Installs PyTorch, PyTorch/XLA, and Torchvision
# Copy this cell into your own notebooks to use PyTorch on Cloud TPUs 
# Warning: this may take a couple minutes to run

import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

VERSION = "20200325"  # @ param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3727  100  3727    0     0  23588      0 --:--:-- --:--:-- --:--:-- 23588
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200325 ...
Uninstalling torch-1.5.0a0+d6149a7:
  Successfully uninstalled torch-1.5.0a0+d6149a7
Uninstalling torchvision-0.6.0a0+3c254fb:
  Successfully uninstalled torchvision-0.6.0a0+3c254fb
Copying gs://tpu-pytorch/wheels/torch-nightly+20200325-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 83.4 MiB/ 83.4 MiB]                                                
Operation completed over 1 objects/83.4 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200325-cp36-cp36m-linux_x86_64.whl...
- [1 files][114.5 MiB/114.5 MiB]                                                
Operation completed over 1 objects/114.5 MiB.               

In [0]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp

In [0]:
!pip install transformers
import numpy as np
import pandas as pd
import os
import tokenizers
import string
import transformers
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
import re
from sklearn import model_selection
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import time

In [0]:
KAGGLE_INPUT_PATH = "../input/"
COLAB_INPUT_PATH = "/content/drive/My Drive/input/"
INPUT_PATH = COLAB_INPUT_PATH

DATA_PATH = INPUT_PATH + "tweet-sentiment-extraction/"
CHECKPOINT_PATH = INPUT_PATH + "model_checkpoint/roberta/"
CHECKPOINT_PATH_TORCH = INPUT_PATH + "model_checkpoint/roberta/torch/"
# ROBERTA_PATH = INPUT_PATH + 'models/output_twitter/'
ROBERTA_PATH = INPUT_PATH + 'models/roberta-base/'

VER='tpu-tmp'

In [0]:
def print_to_file(text):
  try:
    f = open(CHECKPOINT_PATH_TORCH + VER + "-logs.txt", 'a')
  except:
    f = open(CHECKPOINT_PATH_TORCH + VER + "-logs.txt", 'w')

  print(text)
  f.write(time.strftime("%Y%m%d-%H%M%S"))
  f.write("  -- ")
  f.write(text)
  f.write("\n")

class Timer(object):
    def __init__(self, name=None):
        self.name = name

    def __enter__(self):
        self.tstart = time.time()

    def __exit__(self, type, value, traceback):
        if self.name:
            print_to_file('[%s]' % self.name,)
        print_to_file('Elapsed: %s' % (time.time() - self.tstart))

def jaccard(str1, str2): 

    a = set(str(str1).lower().split()) 
    b = set(str(str2).lower().split())

    c = a.intersection(b)

    if (len(a) + len(b) - len(c)) > 0:
      return float(len(c)) / (len(a) + len(b) - len(c))
    else:
      return 0

In [0]:
class config():
    MAX_LEN = 192
    TRAIN_BATCH_SIZE = 64
    VALID_BATCH_SIZE = 32
    EPOCHS = 3
    TOKENIZER = tokenizers.ByteLevelBPETokenizer(
        vocab_file=f"{ROBERTA_PATH}/vocab.json", 
        merges_file=f"{ROBERTA_PATH}/merges.txt", 
        lowercase=True,
        add_prefix_space=True
    )
    LR = 4e-5

In [0]:
class TweetModel(transformers.BertPreTrainedModel):
    def __init__(self, conf):
        super(TweetModel, self).__init__(conf)
        self.roberta = transformers.RobertaModel.from_pretrained(ROBERTA_PATH, config=conf)
        self.drop_out = nn.Dropout(0.1)
        self.l0 = nn.Linear(768 * 2, 2)
        torch.nn.init.normal_(self.l0.weight, std=0.02)
    
    def forward(self, ids, mask, token_type_ids):
        _, _, out = self.roberta(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids
        )

        out = torch.cat((out[-1], out[-2]), dim=-1)
        out = self.drop_out(out)
        logits = self.l0(out)

        start_logits, end_logits = logits.split(1, dim=-1)

        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        return start_logits, end_logits

In [0]:
def process_data(tweet, selected_text, sentiment, tokenizer, max_len):
    tweet = " " + " ".join(str(tweet).split())
    selected_text = " " + " ".join(str(selected_text).split())

    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None

    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind+len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break

    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    tok_tweet = tokenizer.encode(tweet)
    input_ids_orig = tok_tweet.ids
    tweet_offsets = tok_tweet.offsets
    
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]

    sentiment_id = {
        'positive': 1313,
        'negative': 2430,
        'neutral': 7974
    }
    
    input_ids = [0] + [sentiment_id[sentiment]] + [2] + [2] + input_ids_orig + [2]
    token_type_ids = [0, 0, 0, 0] + [0] * (len(input_ids_orig) + 1)
    mask = [1] * len(token_type_ids)
    tweet_offsets = [(0, 0)] * 4 + tweet_offsets + [(0, 0)]
    targets_start += 4
    targets_end += 4

    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([1] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        tweet_offsets = tweet_offsets + ([(0, 0)] * padding_length)
    
    return {
        'ids': input_ids,
        'mask': mask,
        'token_type_ids': token_type_ids,
        'targets_start': targets_start,
        'targets_end': targets_end,
        'orig_tweet': tweet,
        'orig_selected': selected_text,
        'sentiment': sentiment,
        'offsets': tweet_offsets
    }


class TweetDataset:
    def __init__(self, tweet, sentiment, selected_text):
        self.tweet = tweet
        self.sentiment = sentiment
        self.selected_text = selected_text
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_LEN
    
    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, item):
        data = process_data(
            self.tweet[item], 
            self.selected_text[item], 
            self.sentiment[item],
            self.tokenizer,
            self.max_len
        )

        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
            'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long),
            'targets_start': torch.tensor(data["targets_start"], dtype=torch.long),
            'targets_end': torch.tensor(data["targets_end"], dtype=torch.long),
            'orig_tweet': data["orig_tweet"],
            'orig_selected': data["orig_selected"],
            'sentiment': data["sentiment"],
            'offsets': torch.tensor(data["offsets"], dtype=torch.long)
        }

In [0]:
def calculate_jaccard_score(
    original_tweet, 
    target_string, 
    sentiment_val, 
    idx_start, 
    idx_end, 
    offsets,
    verbose=False):
    
    if idx_end < idx_start:
        idx_end = idx_start
    
    filtered_output  = ""
    for ix in range(idx_start, idx_end + 1):
        filtered_output += original_tweet[offsets[ix][0]: offsets[ix][1]]
        if (ix+1) < len(offsets) and offsets[ix][1] < offsets[ix+1][0]:
            filtered_output += " "

    if len(original_tweet.split()) < 2:
        filtered_output = original_tweet

    if sentiment_val != "neutral" and verbose == True:
        if filtered_output.strip().lower() != target_string.strip().lower():
            print("********************************")
            print(f"Output= {filtered_output.strip()}")
            print(f"Target= {target_string.strip()}")
            print(f"Tweet= {original_tweet.strip()}")
            print("********************************")

    jac = jaccard(target_string.strip(), filtered_output.strip())
    return jac, filtered_output

In [0]:
def load_data(train_df, train_idx, valid_idx):
    df_train = train_df.iloc[train_idx]
    df_valid = train_df.iloc[valid_idx]

    train_dataset = TweetDataset(
        tweet=df_train.text.values,
        sentiment=df_train.sentiment.values,
        selected_text=df_train.selected_text.values
    )

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.TRAIN_BATCH_SIZE,
        num_workers=4
    )

    valid_dataset = TweetDataset(
        tweet=df_valid.text.values,
        sentiment=df_valid.sentiment.values,
        selected_text=df_valid.selected_text.values
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=config.VALID_BATCH_SIZE,
        num_workers=2
    )

    return train_data_loader, valid_data_loader, df_train, df_valid
        
def loss_fn(start_logits, end_logits, start_positions, end_positions):
    loss_fct = nn.CrossEntropyLoss()
    start_loss = loss_fct(start_logits, start_positions)
    end_loss = loss_fct(end_logits, end_positions)
    total_loss = (start_loss + end_loss)
    return total_loss

def calculate_valid_score(device, model, valid_data_loader):
    with torch.no_grad():
        model.eval()
        final_output = []
        jaccard_scores = []

        for d in tqdm(valid_data_loader, total=len(valid_data_loader), position=0, leave=True):
            outputs_start, outputs_end = model(
                ids=d["ids"].to(device, dtype=torch.long),
                mask=d["mask"].to(device, dtype=torch.long),
                token_type_ids=d["token_type_ids"].to(device, dtype=torch.long)
            )
            outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
            outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()

            for px in range(len(d["offsets"])):

                targets_start = d["targets_start"] [px]
                targets_end = d["targets_end"][px]
                offsets = d["offsets"][px]
                
                
                jac, output_sentence = calculate_jaccard_score(
                    original_tweet=d["orig_tweet"][px],
                    target_string=d["orig_selected"][px],
                    sentiment_val=d["sentiment"][px],
                    idx_start=np.argmax(outputs_start[px, :]),
                    idx_end=np.argmax(outputs_end[px, :]),
                    offsets=d["offsets"][px]
                )
                final_output.append(output_sentence)
                jaccard_scores.append(jac)
        return np.mean(jaccard_scores), jaccard_scores, final_output

In [0]:
def train_model(device, train_data_loader, valid_data_loader, fold_num, num_epochs, num_train_steps, train_jac_metric=False):
    """
    training model for a specific fold
    includes validation on oof set and saving the best model
    """
    model_config = transformers.RobertaConfig.from_pretrained(ROBERTA_PATH)
    model_config.output_hidden_states = True
    model = TweetModel(conf=model_config).to(device)

    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    ]
    optimizer = AdamW(optimizer_parameters, lr=config.LR)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=0, 
        num_training_steps=num_train_steps
    )
    best_score = -100
    for epoch in range(num_epochs):
        model.zero_grad()
        model.train()
        print_to_file("Fold %d, Epoch %d" % (fold_num+1, epoch+1))
        time.sleep(1) 
        fold_jaccard_scores = []
        for d in tqdm(train_data_loader, total=len(train_data_loader), position=0, leave=True):
            
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            targets_start = d["targets_start"]
            targets_end = d["targets_end"]
            sentiment = d["sentiment"]
            orig_selected = d["orig_selected"]
            orig_tweet = d["orig_tweet"]
            targets_start = d["targets_start"]
            targets_end = d["targets_end"]
            offsets = d["offsets"]

            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets_start = targets_start.to(device, dtype=torch.long)
            targets_end = targets_end.to(device, dtype=torch.long)

            model.zero_grad()
            outputs_start, outputs_end = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids,
            )
            loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
            loss.backward()
            # optimizer.step()
            xm.optimizer_step(optimizer, barrier=True, optimizer_args={}, groups=[[i] for i in range(8)])
            scheduler.step()

        valid_jaccard_score, oof_jaccard_scores, oof_final_output = calculate_valid_score(device, model, valid_data_loader)

        print_to_file("%i | Jaccard average - valid: "%(fold_num+1) + str(valid_jaccard_score)) 
        if valid_jaccard_score > best_score:
            best_score = valid_jaccard_score
            xm.save(model.state_dict(), CHECKPOINT_PATH_TORCH + '%s-roberta-%i.bin'%(VER,fold_num+1),master_only=False)
        else:
            print_to_file("%i | Jaccard average didn't improve"%(fold_num+1))

    return best_score, model, oof_jaccard_scores, oof_final_output

In [0]:
train_df = pd.read_csv(DATA_PATH + 'train.csv').dropna().reset_index().sample(n=2000).reset_index()

df_test = pd.read_csv(DATA_PATH + 'test.csv')
df_test.loc[:, "selected_text"] = df_test.text.values

In [0]:
train_data_loader_list = []
valid_data_loader_list = []
df_train_list = []
num_train_steps_list = []
num_folds = 8

kfold = model_selection.KFold(n_splits=num_folds, shuffle=True, random_state=42)
for fold_num, (train_idx, valid_idx) in enumerate(kfold.split(train_df.text)):
    # if (fold_num < 1):
    train_data_loader, valid_data_loader, df_train, _ = load_data(train_df, train_idx, valid_idx)
    num_train_steps = int(len(df_train) / config.TRAIN_BATCH_SIZE * config.EPOCHS)

    train_data_loader_list.append(train_data_loader)    
    valid_data_loader_list.append(valid_data_loader)    
    df_train_list.append(df_train)
    num_train_steps_list.append(num_train_steps)  
    # else:
    #     train_data_loader_list.append(train_data_loader_list[0])    
    #     valid_data_loader_list.append(valid_data_loader_list[0])    
    #     df_train_list.append(df_train_list[0])
    #     num_train_steps_list.append(num_train_steps_list[0]) 

In [0]:
import warnings
warnings.filterwarnings('ignore')

def run_fold(i, flags):
    device = xm.xla_device()  
    print_to_file("Fold %i is using " % (i) + str(xm.xla_real_devices([str(device)])[0]))

    best_score = 0
    best_score = train_model(
        device, 
        train_data_loader_list[i], 
        valid_data_loader_list[i], 
        fold_num=i, 
        num_epochs=3,
        num_train_steps=num_train_steps_list[i]
        )

    # Barrier to prevent master from exiting before workers connect.
    xm.rendezvous('init')

    return best_score

flags = {}
# Note: Colab only supports start_method='fork'
xmp.spawn(run_fold, args=(flags,), nprocs=8, start_method='fork')

Fold 1 is using TPU:1
Fold 2 is using TPU:2
Fold 3 is using TPU:3
Fold 4 is using TPU:4
Fold 5 is using TPU:5
Fold 6 is using TPU:6
Fold 7 is using TPU:7
Fold 4, Epoch 1
Fold 5, Epoch 1
Fold 8, Epoch 1


  0%|          | 0/28 [00:00<?, ?it/s]

Fold 7, Epoch 1
Fold 6, Epoch 1
Fold 3, Epoch 1


  0%|          | 0/28 [00:00<?, ?it/s]
Exception in device=TPU:3: optimizer_step() got an unexpected keyword argument 'groups'
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 119, in _start_fn
    fn(gindex, *args)
  File "<ipython-input-40-bb2626eb36b5>", line 15, in run_fold
    num_train_steps=num_train_steps_list[i]
  File "<ipython-input-37-d07a8ca5b1a2>", line 58, in train_model
    xm.optimizer_step(optimizer, barrier=True,groups=[[i] for i in range(8)])
TypeError: optimizer_step() got an unexpected keyword argument 'groups'
  0%|          | 0/28 [00:00<?, ?it/s]
Exception in device=TPU:4: optimizer_step() got an unexpected keyword argument 'groups'
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 119, in _start_fn
    fn(gindex, *args)
  File "<ipython-input-40-bb2626eb36b5>", line 15, in run_fold
    num_trai

Exception: ignored